# ETL Project

### Group Project by:

##### Bay Rodriguez  /  Chris Tran  /  Emil Rodulfo  /  Tim Besaw

In [35]:
# Dependencies and Setup
import pandas as pd
from sqlalchemy import create_engine
import sys

sys.path.append("static/assets/Resources/")
import config

## Extract Data

In [36]:
# Load and read game file
game_file = 'static/assets/Resources/game_stats.csv'
game_df = pd.read_csv(game_file, encoding='utf-8')

# Minimize data to bowl games from 2007-2016 and filter down fields pertinent to analysis
new_game_df = game_df.loc[game_df['Year'] >2006, :]
new_game_df = new_game_df[['Year', 'Away Team', 'Home Team', 'Bowl', 'id', 'Home Bowl Scores', 'Away Bowl Scores', 'Winning Team', 'Losing Team']]

new_game_df.head()

,Year,Away Team,Home Team,Bowl,id,Home Bowl Scores,Away Bowl Scores,Winning Team,Losing Team
1,2011,Washington Huskies,Baylor Bears,Alamo Bowl,17746,67,56,Baylor Bears,Washington Huskies
2,2016,Idaho Vandals,Colorado State Rams,Idaho Potato Bowl,27073,50,61,Idaho Vandals,Colorado State Rams
3,2015,Toledo Rockets,Arkansas State Red Wolves,GoDaddy Bowl,23972,44,63,Toledo Rockets,Arkansas State Red Wolves
4,2015,Tulsa Golden Hurricane,Virginia Tech Hokies,Independence Bowl,25533,55,52,Virginia Tech Hokies,Tulsa Golden Hurricane
5,2014,BYU Cougars,Memphis Tigers,Miami Beach Bowl,23939,55,48,Memphis Tigers,BYU Cougars


In [37]:
# Load and read player file
player_file = 'static/assets/Resources/player_stats.csv'
player_df = pd.read_csv(player_file, encoding='utf-8')

# Filter data to fields pertinent to analysis
new_player_df=player_df[['GameId','team','player']]

new_player_df.head()

,GameId,team,player
0,27067,UTSA Roadrunners,Sturm
1,27067,New Mexico Lobos,Jordan
2,27067,New Mexico Lobos,Apodaca
3,27067,UTSA Roadrunners,Williams
4,27067,UTSA Roadrunners,Rhodes


## Transform Data

In [38]:
# Rename game dataframe columns to match database field names
xform_game = new_game_df.rename(columns={'Year': 'year', 'Away Team': 'away_team', 'Home Team': 'home_team', 'Bowl': 'bowl', 'id': 'gameid', 'Home Bowl Scores': 'home_bowl_scores', 'Away Bowl Scores': 'away_bowl_scores', 'Winning Team': 'winning_team', 'Losing Team': 'losing_team'})

# Set index
xform_game.set_index('gameid', inplace=True)

xform_game.head(1000)


,year,away_team,home_team,bowl,home_bowl_scores,away_bowl_scores,winning_team,losing_team
gameid,,,,,,,,
17746,2011,Washington Huskies,Baylor Bears,Alamo Bowl,67,56,Baylor Bears,Washington Huskies
27073,2016,Idaho Vandals,Colorado State Rams,Idaho Potato Bowl,50,61,Idaho Vandals,Colorado State Rams
23972,2015,Toledo Rockets,Arkansas State Red Wolves,GoDaddy Bowl,44,63,Toledo Rockets,Arkansas State Red Wolves
25533,2015,Tulsa Golden Hurricane,Virginia Tech Hokies,Independence Bowl,55,52,Virginia Tech Hokies,Tulsa Golden Hurricane
23939,2014,BYU Cougars,Memphis Tigers,Miami Beach Bowl,55,48,Memphis Tigers,BYU Cougars
17763,2012,West Virginia Mountaineers,Clemson Tigers,Orange Bowl,33,70,West Virginia Mountaineers,Clemson Tigers
20823,2013,Duke Blue Devils,Texas A&M Aggies,Chick-fil-A Bowl,52,48,Texas A&M Aggies,Duke Blue Devils
11686,2007,Purdue Boilermakers,Central Michigan Chippewas,Motor City Bowl,48,51,Purdue Boilermakers,Central Michigan Chippewas
23944,2014,Central Michigan Chippewas,Western Kentucky Hilltoppers,Popeyes Bahamas Bowl,49,48,Western Kentucky Hilltoppers,Central Michigan Chippewas


In [39]:
# Rename player dataframe columns to match database field names
xform_player = new_player_df.rename(columns={'GameId':'gameid', 'team':'team', 'player':'player'})

# Set index
xform_player.set_index('gameid', inplace=True)

xform_player.head()

,team,player
gameid,,
27067,UTSA Roadrunners,Sturm
27067,New Mexico Lobos,Jordan
27067,New Mexico Lobos,Apodaca
27067,UTSA Roadrunners,Williams
27067,UTSA Roadrunners,Rhodes


## Establish Database Connection

* Use config.py file to store MySQL credentials, or update them inline

In [40]:
# Create database connection
conn = (f'{config.username}:{config.password}@localhost/football_db')
engine = create_engine(f'mysql://{conn}')

## Run DDL to Build Database Tables

In [41]:
# Create database and supporting tables 

sql = """
drop database if exists football_db;

create database football_db;

use football_db;


CREATE TABLE bowls (
  bowl_id int(11) NOT NULL AUTO_INCREMENT,
  bowl varchar(100) DEFAULT NULL,
  PRIMARY KEY (bowl_id)
) ENGINE=InnoDB AUTO_INCREMENT=128 DEFAULT CHARSET=latin1;

CREATE TABLE teams (
  team_id int(11) NOT NULL AUTO_INCREMENT,
  team varchar(100) DEFAULT NULL,
  PRIMARY KEY (team_id)
) ENGINE=InnoDB AUTO_INCREMENT=256 DEFAULT CHARSET=latin1;


CREATE TABLE games (
  game_id int(11) NOT NULL,
  year int(11) DEFAULT NULL,
  bowl_id int(11) DEFAULT NULL,
  home_team_id int(11) DEFAULT NULL,
  away_team_id int(11) DEFAULT NULL,
  home_score int(11) DEFAULT NULL,
  away_score int(11) DEFAULT NULL,
  PRIMARY KEY (game_id),
  KEY game_fk01_idx (bowl_id),
  KEY game_fk02_idx (home_team_id),
  KEY game_fk03_idx (away_team_id),
  CONSTRAINT game_fk01 FOREIGN KEY (bowl_id) REFERENCES bowls (bowl_id) ON DELETE NO ACTION ON UPDATE NO ACTION,
  CONSTRAINT game_fk02 FOREIGN KEY (home_team_id) REFERENCES teams (team_id) ON DELETE NO ACTION ON UPDATE NO ACTION,
  CONSTRAINT game_fk03 FOREIGN KEY (away_team_id) REFERENCES teams (team_id) ON DELETE NO ACTION ON UPDATE NO ACTION
) ENGINE=InnoDB DEFAULT CHARSET=latin1;


CREATE TABLE import_game_stats (
  gameid int(11) NOT NULL,
  year int(11) DEFAULT NULL,
  away_team varchar(100) DEFAULT NULL,
  home_team varchar(100) DEFAULT NULL,
  bowl varchar(100) DEFAULT NULL,
  home_bowl_scores int(11) DEFAULT NULL,
  away_bowl_scores int(11) DEFAULT NULL,
  winning_team varchar(100) DEFAULT NULL,
  losing_team varchar(100) DEFAULT NULL,
  PRIMARY KEY (gameid)
) ENGINE=InnoDB DEFAULT CHARSET=latin1;


CREATE TABLE import_player_stats (
  playerId int(11) NOT NULL AUTO_INCREMENT,
  gameId int(11) DEFAULT NULL,
  team varchar(100) DEFAULT NULL,
  player varchar(100) DEFAULT NULL,
  PRIMARY KEY (playerId)
) ENGINE=InnoDB AUTO_INCREMENT=19997 DEFAULT CHARSET=latin1;


CREATE TABLE players (
  player_id int(11) NOT NULL AUTO_INCREMENT,
  game_id int(11) DEFAULT NULL,
  team_id int(11) DEFAULT NULL,
  player varchar(100) DEFAULT NULL,
  PRIMARY KEY (player_id),
  KEY players_fk01_idx (game_id),
  KEY players_fk02_idx (team_id),
  CONSTRAINT players_fk01 FOREIGN KEY (game_id) REFERENCES games (game_id) ON DELETE NO ACTION ON UPDATE NO ACTION,
  CONSTRAINT players_fk02 FOREIGN KEY (team_id) REFERENCES teams (team_id) ON DELETE NO ACTION ON UPDATE NO ACTION
) ENGINE=InnoDB AUTO_INCREMENT=13858 DEFAULT CHARSET=latin1;


CREATE TABLE flsk_bowl_history (
  bowl_id int(11) NOT NULL PRIMARY KEY,
  bowl varchar(100) DEFAULT NULL,
  cnt_games bigint(21) NOT NULL DEFAULT '0',
  min_year bigint(11) DEFAULT NULL,
  max_year bigint(11) DEFAULT NULL,
  home_teams text,
  away_teams text
) ENGINE=InnoDB DEFAULT CHARSET=latin1;


CREATE TABLE flsk_bowl_outcome (
  game_id int(11) NOT NULL primary key ,
  year int(11) DEFAULT NULL,
  bowl varchar(100) DEFAULT NULL,
  home_team varchar(100) DEFAULT NULL,
  away_team varchar(100) DEFAULT NULL,
  home_score int(11) DEFAULT NULL,
  away_score int(11) DEFAULT NULL,
  winning_team varchar(100) DEFAULT NULL,
  loosing_team varchar(100) DEFAULT NULL
) ENGINE=InnoDB DEFAULT CHARSET=latin1;

CREATE TABLE flsk_bowl_players (
  player_id bigint(11) NOT NULL PRIMARY KEY,
  year int(11) DEFAULT NULL,
  bowl varchar(100) DEFAULT NULL,
  team varchar(100) DEFAULT NULL,
  player varchar(100) DEFAULT NULL
) ENGINE=InnoDB DEFAULT CHARSET=latin1;

  """

engine.execute(sql)

In [42]:
# Confirm tables
engine.table_names()

['bowls',
 'flsk_bowl_history',
 'flsk_bowl_outcome',
 'flsk_bowl_players',
 'games',
 'import_game_stats',
 'import_player_stats',
 'players',
 'teams']

## Run DDL to Build Database Views

In [43]:
# Create database and supporting views

sql = """

create or replace view football_db.years_vw as
select  distinct year 
from    football_db.games
order by year;


create or replace view football_db.teams_vw as
select  t.team,  ( select count(1) 
                   from   football_db.games g 
                   where  g.home_team_id = t.team_id 
                      or  g.away_team_id = t.team_id ) as cnt_games
from    football_db.teams t
;


create or replace view football_db.bowls_vw as
select  bowl
from    football_db.bowls
;


create or replace view games_vw as
select g.game_id,
       g.year,
       g.bowl_id,
       b.bowl,
       th.team as home_team,
       ta.team as away_team,
       g.home_score,
       g.away_score,
       case when g.home_score > g.away_score then th.team else ta.team end as winning_team,
       case when g.home_score < g.away_score then th.team else ta.team end as loosing_team       
from   football_db.games g
join   football_db.bowls b  on g.bowl_id = b.bowl_id
join   football_db.teams th on g.home_team_id = th.team_id
join   football_db.teams ta on g.away_team_id = ta.team_id
;


create or replace view players_vw as
select p.player_id, g.year, b.bowl, t.team, p.player
from   football_db.players p
join   football_db.games   g  on p.game_id = g.game_id
join   football_db.bowls   b  on g.bowl_id = b.bowl_id
join   football_db.teams   t  on p.team_id = t.team_id
;

create or replace view bowl_history_vw as
select bowl_id,
       bowl, 
       count(1) as cnt_games,
       min(year) as min_year,
       max(year) as max_year,
       group_concat(distinct home_team order by home_team separator ', ') home_teams,
       group_concat(distinct away_team order by away_team separator ', ') away_teams       
from games_vw x
group by bowl
order by bowl
;

create or replace view bowl_outcome_vw as
select game_id,
       year,
       bowl, 
       home_team,
       away_team,
       home_score,
       away_score,
       winning_team,
       loosing_team
from games_vw x
order by bowl
;

create or replace view bowl_players_vw as
select  min(player_id) as player_id,
        year,
        bowl,
        team,
        player
from    players_vw 
group by year, bowl, team, player
order by team, player
;

  """

engine.execute(sql)

## Import Data Acquired from CSV files 

In [44]:
# Import game dataframe to MySQL database
xform_game.to_sql(name='import_game_stats', con=engine, if_exists = 'append', index=True)

In [45]:
# Import player dataframe to MySQL database
xform_player.to_sql(name='import_player_stats', con=engine, if_exists = 'append', index=True)

In [46]:
# run sql to delete orphan player records as a result of year filter on games

sql = """ 
  delete  
  from    football_db.import_player_stats
  where   gameid not in ( select gameid from football_db.import_game_stats)
  """

engine.execute(sql)

In [47]:
# run sql to moved staged data into bowls table 

sql = """ 
  insert into football_db.bowls(bowl)
  select distinct bowl
  from   football_db.import_game_stats
  order by bowl
  """
# run delete to remove orphan player records as a result of year filter on games
engine.execute(sql)



In [48]:
# run sql to moved staged data into teams table 

sql = """ 
  insert into football_db.teams(team)
  select home_team as team 
  from   football_db.import_game_stats
  union 
  select away_team as team 
  from   football_db.import_game_stats
  order by team
  """
# run delete to remove orphan player records as a result of year filter on games
engine.execute(sql)

In [49]:
# run sql to moved staged data into games table

sql = """ 
  insert into football_db.games
  select g.gameid as game_id,
         g.year,
         ( select x.bowl_id from bowls x where x.bowl = g.bowl ) as bowl_id,	   
         ( select x.team_id from teams x where x.team = g.home_team ) as home_team_id,
         ( select x.team_id from teams x where x.team = g.away_team ) as away_team_id,
         g.home_bowl_scores as home_score,    
         g.away_bowl_scores as away_score
  from   football_db.import_game_stats g
  """
# run delete to remove orphan player records as a result of year filter on games
engine.execute(sql)

In [50]:
# run sql to moved staged data into players table

sql = """ 
  insert into football_db.players
  select ps.playerid as player_id,
         ps.gameid   as game_id,
         ( select x.team_id from teams x where x.team = ps.team ) as team_id,
         player
  from   football_db.import_player_stats ps
  """

# run delete to remove orphan player records as a result of year filter on games
engine.execute(sql)

In [51]:
# run sql for final presentation tables

sql = """ 
  insert into flsk_bowl_history 
  select * from bowl_history_vw;
  
  insert into flsk_bowl_outcome 
  select * from bowl_outcome_vw;
  
  insert into flsk_bowl_players 
  select * from bowl_players_vw;    
  """

# run delete to remove orphan player records as a result of year filter on games
engine.execute(sql)

### Views to be Referenced in Flask App 

* flsk_bowl_history
* flsk_bowl_outcome
* flsk_bowl_players
